# HIDDEN MARKOV NEURAL NETWORK: MNIST example

## Import libraries

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pickle

import numpy as np

import BayesianNetwork

## Set the hyper parameters for the training

In [ ]:
# The sample size
sample_size    = 10000
# The minibatch size
minibatch_size = 128
# The number of epochs
epocs          = 600

sliding = 10000
T = 5

###########################################################
# Set the network structure

L = 4
architecture = np.array([784, 400, 400, 10])

alpha_k = 0.25
sigma_k = np.exp(2)
c       = np.exp(10)
p       = 0.3
pi      = 0.5
lr_c = 1e-3
mc_c = 1